# Mavlink Component
> Mavlink  base class for Client and server.
> Theserver is on the companion computer and the client is on the ground station PC.

[https://mavlink.io/en/mavgen_python/](https://mavlink.io/en/mavgen_python/)
[https://www.ardusub.com/developers/pymavlink.html](https://www.ardusub.com/developers/pymavlink.html)

https://mavlink.io/en/messages/common.html
https://mavlink.io/en/messages/common.html#MAV_TYPE


In [ ]:
#| default_exp mavlink.component

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
import time, os, sys

from UAV.logging import logging
from UAV.utils.general import LeakyQueue

# os.environ['MAVLINK20'] == '1' should be placed in UAV.__init__.py
assert os.environ[
           'MAVLINK20'] == '1', "Set the environment variable before from pymavlink import mavutil  library is imported"

# logging.getLogger("uav").setLevel(logging.DEBUG)
# logging.root.setLevel(logging.INFO)
import threading
import queue
import typing as typ
from pathlib import Path
from inspect import currentframe, getframeinfo
from pymavlink import mavutil


# from UAV.imports import *   # TODO why is this relative import on nbdev_export?


In [ ]:
# logging.getLogger("uav").setLevel(logging.INFO)

In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from UAV.mavlink.mavcom import MAVCom, BaseComponent, get_linenumber, format_rcvd_msg

In [ ]:
#| export
from UAV.mavlink.mavcom import MAVCom, BaseComponent, get_linenumber, format_rcvd_msg

In [ ]:
#| export
class Component:
    """Create a mavlink Component with an ID  for MAV_COMPONENT"""

    def __init__(self, mav_connection,  # MavLinkBase connection
                 source_component,  # used for component indication
                 mav_type,  # used for heartbeat MAV_TYPE indication
                 debug):  # logging level
        # todo change to def __init__(self:MavLinkBase, ....
        self.mav_connection: MAVCom = mav_connection
        self.master = mav_connection.master
        self.mav_type = mav_type
        self.source_system = self.mav_connection.source_system
        self.source_component = source_component


        self._log = logging.getLogger("uav.{}".format(self.__class__.__name__))
        self._log.setLevel(logging.DEBUG if debug else logging.INFO)

        self.ping_num = 0
        self.max_pings = 4
        self.num_msgs_rcvd = 0
        self.num_cmds_sent = 0
        self.num_cmds_rcvd = 0
        self.num_acks_sent     = 0
        self.num_acks_rcvd = 0
        self.num_acks_drop = 0
        self.message_cnts: {} = {}  # received message counts, indexed by system and message type

        self._heartbeat_que = LeakyQueue(maxsize=10)
        self._ack_que = LeakyQueue(maxsize=10)
        self._message_que = LeakyQueue(maxsize=10)

        self._t_heartbeat = threading.Thread(target=self.send_heartbeat, daemon=True)
        self._t_heartbeat.start()

        self._t_command = threading.Thread(target=self.listen, daemon=True)
        self._t_command.start()
        self.log.info(
            f"Component Started {self.source_component = }, {self.mav_type = }, {self.source_system = }")

    def __str__(self) -> str:
        return self.__class__.__name__

    def __repr__(self) -> str:
        return "<{}>".format(self)

    @property
    def log(self) -> logging.Logger:
        return self._log

    def set_source_compenent(self):
        """Set the source component for the master.mav """
        self.master.mav.srcComponent = self.source_component

    def send_ping(self, target_system: int, target_component: int, ping_num: int = None):
        """Send self.max_pings * ping messages to test if the server is alive."""

        if ping_num == 0:
            self.ping_num = 0
        if self.ping_num >= self.max_pings:
            return

        self.set_source_compenent()
        self.master.mav.ping_send(
            int(time.time() * 1000),  # Unix time 
            self.ping_num,  # Ping number
            target_system,  # Request ping of this system
            target_component,  # Request ping of this component
        )
        self.log.debug(f"Sent Ping #{self.ping_num} to:   {target_system:3d}, comp: {target_component:3d}")
        self.ping_num += 1

    def send_heartbeat(self):
        """Send a heartbeat message to indicate the server is alive."""
        self._t_heartbeat_stop = False

        # self.log.info(f"Starting heartbeat type: {self.mav_type} to all Systems and Components")
        while not self._t_heartbeat_stop:
            self.set_source_compenent()
            # self.log.debug(f"Sent hrtbeat to All")
            # "Sent Ping #2 to:   111, comp: 100"
            self.master.mav.heartbeat_send(
                self.mav_type,  # type
                # mavutil.mavlink.MAV_TYPE_ONBOARD_CONTROLLER,
                mavutil.mavlink.MAV_AUTOPILOT_INVALID,  # autopilot
                0,  # base_mode
                0,  # custom_mode
                mavutil.mavlink.MAV_STATE_ACTIVE,  # system_status
            )
            # print("Cam heartbeat_send")
            time.sleep(1)  # Send every second

    def wait_heartbeat(self, remote_mav_type=None,  # type of remote system
                       target_system=None,  # target system
                       target_component=None,  # target component
                       timeout: int = 1,  # seconds
                       tries: int = 3) -> bool:  # number of tries
        """Wait for a heartbeat from target_system and target_component."""
        # Todo is this correct ? Wait for a heartbeat, so we know the target system IDs (also it seems to need it to start receiving commands)
        if remote_mav_type is None:
            self.log.debug(f"Waiting for heartbeat from {target_system = }:  {target_component = }")
        else:
            self.log.debug(
                f"Waiting for heartbeat from {remote_mav_type} from {target_system = }:  {target_component = }")
        count = 0
        while count < tries:
            try:
                msg = self._heartbeat_que.get(timeout=timeout)
                self.log.debug(format_rcvd_msg(msg, extra='self._heartbeat_que.get() '))
                # self.log.debug(f"Rcvd Heartbeat from src_sys: {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg} ")
                # check if the heartbeat is from the correct system and component
                if msg.type == remote_mav_type and msg.get_srcSystem() == target_system and msg.get_srcComponent() == target_component:
                    return True
                elif msg.get_srcSystem() == target_system and msg.get_srcComponent() == target_component:
                    return True
            except queue.Empty:  # i.e time out
                count += 1

        self.log.debug(f"No heartbeat received after {tries} tries")
        return False

    def send_ack(self, msg, ack_result: object = mavutil.mavlink.MAV_RESULT_ACCEPTED):
        """Send an ACK message to indicate a command was received."""
        self.set_source_compenent()
        self.master.mav.command_ack_send(
            msg.command,
            ack_result,  # or other MAV_RESULT enum
            # todo enabling these causes QGC not to show them
            int(0),  # progress
            int(0),  # result_param2
            msg.get_srcSystem(),  # target_system = msg.get_srcSystem(),  # target_system
            msg.get_srcComponent(),  # target_component = msg.get_srcComponent(),  # target_component
        )
        self.log.debug(f"Sent ACK for command: {msg.command} to system: {msg.get_srcSystem()} comp: {msg.get_srcComponent()}")
        self.num_acks_sent += 1

    def _wait_ack(self, target_system, target_component, command_id=None, timeout = 0.1) -> bool:
        """Wait for an ack from target_system and target_component."""
        self.log.debug(f"Waiting for ACK for command: {command_id} from system: {target_system} comp: {target_component}")

        try:
            msg = self._ack_que.get(timeout=timeout)
            # self.log.debug(f"ACK received from src_sys: {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg}")
            if (command_id == msg.command or command_id is None)  and msg.get_srcSystem() == target_system and msg.get_srcComponent() == target_component:
                self.log.debug(f"*** ACK received for cmd {command_id} from src_comp: {msg.get_srcComponent()} {msg}")
                return True
            else:
                self.log.debug(f"*** ACK not handled {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg}")
                self.log.debug(f"{command_id = } {msg.get_srcSystem() = }, {target_system = },  {msg.get_srcComponent() = }, {target_component = }")
        except queue.Empty:  # i.e time out
            pass
        self.log.debug("*** No ACK received")
        return False

    def wait_ack(self, target_system, target_component, command_id=None, timeout = 0.1) -> bool:
        """Wait for an ack from target_system and target_component."""
        self.log.debug(f"Waiting for ACK for command: {command_id} from system: {target_system} comp: {target_component}")
        _time = 0
        _TIME_STEP = 0.1
        while _time < timeout:
            _time += _TIME_STEP
            # print(f"{_time = }")
            try:
                msg = self._ack_que.get(timeout=_TIME_STEP)
                # self.log.debug(f"ACK received from src_sys: {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg}")
                if (command_id == msg.command or command_id is None)  and msg.get_srcSystem() == target_system and msg.get_srcComponent() == target_component:
                    self.log.debug(f"*** ACK received for cmd {command_id} from src_comp: {msg.get_srcComponent()} {msg}")
                    return True
                else:
                    self.log.debug(f"*** ACK not handled {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg}")
                    print(f"{command_id = } {msg.get_srcSystem() = }, {target_system = },  {msg.get_srcComponent() = }, {target_component = }")
                    self.log.debug(f"{command_id = } {msg.get_srcSystem() = }, {target_system = },  {msg.get_srcComponent() = }, {target_component = }")

            except queue.Empty:  # i.e time out
                pass

        self.log.debug("!!!!*** No ACK received")
        return False
    def count_message(self, msg):
        """ Count a message by adding it to the message_cnts dictionary. indexed by system and message type"""
        try:
            self.message_cnts[msg.get_srcSystem()][msg.get_type()] += 1
        except Exception as e:
            # print(f"!!!! new Message type {msg.get_type()} from system {msg.get_srcSystem()}")
            sys = msg.get_srcSystem()
            if sys not in self.message_cnts:
                self.message_cnts[sys] = {}
            self.message_cnts[sys][msg.get_type()] = 1

        return True

    def listen(self, timeout: int = 1, ):  # seconds
        """Listen for MAVLink commands and trigger the camera when needed."""

        self._t_cmd_listen_stop = False
        # self.log.info(f"Component Listening for messages sent on the message_queue ...")
        while not self._t_cmd_listen_stop:

            try:
                msg = self._message_que.get(timeout=timeout)
                if msg.get_type() != 'HEARTBEAT':
                    self.log.debug(format_rcvd_msg(msg))
                self.num_msgs_rcvd += 1
            except queue.Empty:  # i.e time out
                time.sleep(0.01)
                continue

            self.count_message(msg)

            # print (f"{msg.get_type() = }")
            if msg.get_type() == 'COMMAND_LONG':
                # print("Om command ")
                self.on_command_rcvd(msg)
            elif msg.get_type() == 'COMMAND_INT':
                self.on_command_rcvd(msg)

            elif msg.get_type() == 'COMMAND_ACK':
                self.log.debug(f"Received ACK ")
                self._ack_que.put(msg, block=False)

            elif msg.get_type() == 'HEARTBEAT':
                # self.log.debug(f"Received HEARTBEAT ")
                self._heartbeat_que.put(msg, block=False)

            elif msg.get_type() == 'PING':
                # self.log.debug(f"Received PING {msg}")
                # ping_num = msg.time_usec
                ping_num = msg.seq
                # print(f"{ping_num = } {msg}")
                if ping_num < self.max_pings:
                    self.log.debug(f"Received PING {msg}")
                    self.send_ping(msg.get_srcSystem(), msg.get_srcComponent())

    def on_command_rcvd(self, msg):
        # Callback for when a command is received.
        print(f"!!! YAY!!! {get_linenumber()} {self} Received command {msg}, sending ACK")
        self.num_cmds_rcvd += 1
        self.send_ack(msg, mavutil.mavlink.MAV_RESULT_ACCEPTED)

    def send_command(self, target_system: int,  # target system
                     target_component: int,  # target component
                     command_id: int,  # mavutil.mavlink.MAV_CMD....
                     params: list,  # list of parameters
                     timeout = 0.5,  # seconds
                     ):
        self.log.debug(f"Sending command: {command_id} to system: {target_system} comp: {target_component}")
        self.set_source_compenent()
        self.master.mav.command_long_send(
            target_system,  # target_system   Todo Tried using self.master.target_system but it didn't work
            target_component,  # target_component Todo tried using self.master.target_component but it didn't work
            command_id,  # command id
            0,  # confirmation
            *params  # command parameters
        )
        self.num_cmds_sent += 1

        if self.wait_ack(target_system, target_component, command_id=command_id, timeout=timeout):
            self.log.debug(
                f"ACK received for command: {command_id} from system: {target_system} comp: {target_component}")
            self.num_acks_rcvd += 1
            return True
        else:
            self.log.debug(
                f"No ACK received for command: {command_id} from system: {target_system} comp: {target_component}")
            self.num_acks_drop += 1
            return False

    def _test_command(self, target_system: int,  # target system
                      target_component: int,  # target component
                      camera_id: int = 1):  # camera id (0 for all cams)
        """
        Use MAV_CMD_DO_DIGICAM_CONTROL to trigger a camera 
        """
        self.set_source_compenent()
        mav_cmd = mavutil.mavlink.MAV_CMD_DO_DIGICAM_CONTROL
        rst = self.send_command(target_system, target_component,
                                mav_cmd,
                                [camera_id,  # param1 (session)  or cam # (0 for all cams)
                                 1,  # param2 (trigger capture)
                                 0,  # param3 (zoom pos)
                                 0,  # param4 (zoom step)
                                 0,  # param5 (focus lock)
                                 0,  # param6 (shot ID)
                                 0,  # param7 (command ID)
                                 ])

        # self.log.debug(f"Sent message to:   {target_system:3d}, comp: {target_component:3d} command MAV_CMD_DO_DIGICAM_CONFIGURE")
        return rst

    def close(self):
        self._t_heartbeat_stop = True
        self._t_heartbeat.join()
        self.log.info(f"{self.__class__.__name__} closed")




In [ ]:
# | hide
# assert False, "Stop here"

In [ ]:
show_doc(Component)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#L36){target="_blank" style="float:right; font-size:smaller"}

### Component

>      Component (mav_connection, source_component, mav_type, debug)

Create a mavlink Component with an ID  for MAV_COMPONENT

|    | **Details** |
| -- | ----------- |
| mav_connection | MavLinkBase connection |
| source_component | used for component indication |
| mav_type | used for heartbeat MAV_TYPE indication |
| debug | logging level |

#### Test locally using UDP ports

#### Starting a client and server
 > on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`

In [ ]:
from UAV.mavlink.mavcom import MAVCom
from UAV.mavlink.component import Component, mavutil
import time

MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA

class Cam1(Component):
    def __init__(self,mav_connection, source_component, mav_type, debug=False):
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)

class Cam2(Component):
    def __init__(self,mav_connection, source_component, mav_type, debug=False):
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)
class Cli(Component):
    def __init__(self,mav_connection, source_component, mav_type, debug=False):
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)

In [ ]:
# Test sending a command and receiving an ack from client to server
with MAVCom("udpin:localhost:14445", source_system=111, debug=False) as client:
    with MAVCom("udpout:localhost:14445", source_system=222, debug=False) as server:
        client.add_component(Cli(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 23, debug=False))
        
        for key, comp in client.component.items():
            if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                print ("*** Received heartbeat **** " )
        NUM_TO_SEND = 2
        for i in range(NUM_TO_SEND):
            client.component[11]._test_command(222, 22, 1)
            client.component[11]._test_command(222, 23, 1)
            
        client.component[11]._test_command(222, 24, 1)

    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    print()
    print(f"{client.source_system = } \n{client.summary()} \n")
    print(f"{server.source_system = } \n{server.summary()} \n")

    assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
    assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
    assert client.component[11].num_acks_drop == 1
    assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
    assert server.component[23].num_cmds_rcvd == NUM_TO_SEND

INFO   | uav.MAVCom      | 02.649 |  mavcom.py:368 | Thread-21 (listen) | MAVLink Mav2: True, source_system: 111
INFO   | uav.MAVCom      | 02.750 |  mavcom.py:368 | Thread-22 (listen) | MAVLink Mav2: True, source_system: 222
INFO   | uav.Cli         | 02.752 | component.py: 73 | MainThread         | Component Started self.source_component = 11, self.mav_type = 6, self.source_system = 111
INFO   | uav.Cam1        | 02.752 | component.py: 73 | MainThread         | Component Started self.source_component = 22, self.mav_type = 30, self.source_system = 222
INFO   | uav.Cam1        | 02.753 | component.py: 73 | MainThread         | Component Started self.source_component = 23, self.mav_type = 30, self.source_system = 222
ERROR  | uav.MAVCom      | 02.756 |  mavcom.py:391 | Thread-22 (listen) |  Component 24 does not exist? ; Exception: 24


Check to see if comp.source_component = 11 already exists
Check to see if comp.source_component = 22 already exists
Check to see if comp.source_component = 23 already exists
*** Received heartbeat **** 
!!! YAY!!! mavcom.py:271 Cam1 Received command COMMAND_LONG {target_system : 222, target_component : 22, command : 203, confirmation : 0, param1 : 1.0, param2 : 1.0, param3 : 0.0, param4 : 0.0, param5 : 0.0, param6 : 0.0, param7 : 0.0}, sending ACK
!!! YAY!!! mavcom.py:271 Cam1 Received command COMMAND_LONG {target_system : 222, target_component : 23, command : 203, confirmation : 0, param1 : 1.0, param2 : 1.0, param3 : 0.0, param4 : 0.0, param5 : 0.0, param6 : 0.0, param7 : 0.0}, sending ACK
!!! YAY!!! mavcom.py:271 Cam1 Received command COMMAND_LONG {target_system : 222, target_component : 22, command : 203, confirmation : 0, param1 : 1.0, param2 : 1.0, param3 : 0.0, param4 : 0.0, param5 : 0.0, param6 : 0.0, param7 : 0.0}, sending ACK
!!! YAY!!! mavcom.py:271 Cam1 Received command COM

INFO   | uav.Cam1        | 03.758 | component.py:328 | MainThread         | Cam1 closed
INFO   | uav.Cam1        | 04.761 | component.py:328 | MainThread         | Cam1 closed
INFO   | uav.MAVCom      | 04.762 |  mavcom.py:409 | MainThread         | MAVCom  closed


server.source_system = 222;  server.message_cnts = {111: {'COMMAND_LONG': 5, 'HEARTBEAT': 1}}
client.source_system = 111;  client.message_cnts = {222: {'HEARTBEAT': 3, 'COMMAND_ACK': 4}}

client.source_system = 111 
 - comp.source_component = 11
 - comp.num_msgs_rcvd = 7
 - comp.num_cmds_sent = 5
 - comp.num_cmds_rcvd = 0
 - comp.num_acks_rcvd = 4
 - comp.num_acks_sent = 0
 - comp.num_acks_drop = 1
 - comp.message_cnts = {222: {'HEARTBEAT': 3, 'COMMAND_ACK': 4}} 

server.source_system = 222 
 - comp.source_component = 22
 - comp.num_msgs_rcvd = 3
 - comp.num_cmds_sent = 0
 - comp.num_cmds_rcvd = 2
 - comp.num_acks_rcvd = 0
 - comp.num_acks_sent = 2
 - comp.num_acks_drop = 0
 - comp.message_cnts = {111: {'COMMAND_LONG': 2, 'HEARTBEAT': 1}}
 - comp.source_component = 23
 - comp.num_msgs_rcvd = 3
 - comp.num_cmds_sent = 0
 - comp.num_cmds_rcvd = 2
 - comp.num_acks_rcvd = 0
 - comp.num_acks_sent = 2
 - comp.num_acks_drop = 0
 - comp.message_cnts = {111: {'COMMAND_LONG': 2, 'HEARTBEAT': 1}}

INFO   | uav.Cli         | 06.769 | component.py:328 | MainThread         | Cli closed
INFO   | uav.MAVCom      | 06.770 |  mavcom.py:409 | MainThread         | MAVCom  closed


In [ ]:
#| Hide
# assert False, "Stop here"

In [ ]:
show_doc(Component.set_source_compenent)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Component.set_source_compenent

>      Component.set_source_compenent ()

Set the source component for the master.mav

In [ ]:
show_doc(Component.send_heartbeat)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Component.send_heartbeat

>      Component.send_heartbeat ()

Send a heartbeat message to indicate the server is alive.

In [ ]:
show_doc(Component.on_command_rcvd)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Component.on_command_rcvd

>      Component.on_command_rcvd (msg)

In [ ]:
show_doc(Component.send_command)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Component.send_command

>      Component.send_command (target_system:int, target_component:int,
>                              command_id:int, params:list, timeout=0.5)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | int |  | target system |
| target_component | int |  | target component |
| command_id | int |  | mavutil.mavlink.MAV_CMD.... |
| params | list |  | list of parameters |
| timeout | float | 0.5 | seconds |

In [ ]:
show_doc(Component.wait_heartbeat)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Component.wait_heartbeat

>      Component.wait_heartbeat (remote_mav_type=None, target_system=None,
>                                target_component=None, timeout:int=1,
>                                tries:int=3)

Wait for a heartbeat from target_system and target_component.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| remote_mav_type | NoneType | None | type of remote system |
| target_system | NoneType | None | target system |
| target_component | NoneType | None | target component |
| timeout | int | 1 | seconds |
| tries | int | 3 |  |
| **Returns** | **bool** |  | **number of tries** |

In [ ]:
show_doc(Component.wait_ack)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Component.wait_ack

>      Component.wait_ack (target_system, target_component, command_id=None,
>                          timeout=0.1)

Wait for an ack from target_system and target_component.

In [ ]:
show_doc(Component.send_ping)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Component.send_ping

>      Component.send_ping (target_system:int, target_component:int,
>                           ping_num:int=None)

Send self.max_pings * ping messages to test if the server is alive.

#### Test with Serial ports
Test using a Pixhawk connected via telemetry 2 and USB serial ports.
CamClient is set to udpin:localhost:14445 and CamServer is set to udpout:localhost:14435 udpin is so that the client can receive UDP from the mavproxy server at localhost:14445
mavproxy.py --master=/dev/ttyACM1 --baudrate 57600 --out udpout:localhost:14445 mavproxy.py --master=/dev/ttyACM3 --baudrate 57600 --out udpout:localhost:14435

In [ ]:
# Test sending a command and receiving an ack from client to server
with MAVCom("/dev/ttyACM0", source_system=111, debug=False) as client:
    with MAVCom("/dev/ttyUSB0", source_system=222, debug=False) as server:
        client.add_component(Cli(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 23, debug=False))
        
        for key, comp in client.component.items():
            if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                print ("*** Received heartbeat **** " )
        NUM_TO_SEND = 2
        for i in range(NUM_TO_SEND):
            client.component[11]._test_command(222, 22, 1)
            client.component[11]._test_command(222, 23, 1)
            
        client.component[11]._test_command(222, 24, 1)

    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    print()
    print(f"{client.source_system = } \n{client.summary()} \n")
    print(f"{server.source_system = } \n{server.summary()} \n")

    assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
    assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
    assert client.component[11].num_acks_drop == 1
    assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
    assert server.component[23].num_cmds_rcvd == NUM_TO_SEND

INFO   | uav.MAVCom      | 59.595 |  mavcom.py:368 | Thread-29 (listen) | MAVLink Mav2: True, source_system: 111
INFO   | uav.MAVCom      | 59.714 |  mavcom.py:368 | Thread-30 (listen) | MAVLink Mav2: True, source_system: 222
INFO   | uav.Cli         | 59.715 | component.py: 73 | MainThread         | Component Started self.source_component = 11, self.mav_type = 6, self.source_system = 111
INFO   | uav.Cam1        | 59.717 | component.py: 73 | MainThread         | Component Started self.source_component = 22, self.mav_type = 30, self.source_system = 222
INFO   | uav.Cam1        | 59.718 | component.py: 73 | MainThread         | Component Started self.source_component = 23, self.mav_type = 30, self.source_system = 222
ERROR  | uav.MAVCom      | 59.868 |  mavcom.py:391 | Thread-30 (listen) |  Component 24 does not exist? ; Exception: 24


Check to see if comp.source_component = 11 already exists
Check to see if comp.source_component = 22 already exists
Check to see if comp.source_component = 23 already exists
*** Received heartbeat **** 
!!! YAY!!! mavcom.py:271 Cam1 Received command COMMAND_LONG {target_system : 222, target_component : 22, command : 203, confirmation : 0, param1 : 1.0, param2 : 1.0, param3 : 0.0, param4 : 0.0, param5 : 0.0, param6 : 0.0, param7 : 0.0}, sending ACK
!!! YAY!!! mavcom.py:271 Cam1 Received command COMMAND_LONG {target_system : 222, target_component : 23, command : 203, confirmation : 0, param1 : 1.0, param2 : 1.0, param3 : 0.0, param4 : 0.0, param5 : 0.0, param6 : 0.0, param7 : 0.0}, sending ACK
!!! YAY!!! mavcom.py:271 Cam1 Received command COMMAND_LONG {target_system : 222, target_component : 22, command : 203, confirmation : 0, param1 : 1.0, param2 : 1.0, param3 : 0.0, param4 : 0.0, param5 : 0.0, param6 : 0.0, param7 : 0.0}, sending ACK
!!! YAY!!! mavcom.py:271 Cam1 Received command COM

INFO   | uav.Cam1        | 00.721 | component.py:328 | MainThread         | Cam1 closed
INFO   | uav.Cam1        | 01.725 | component.py:328 | MainThread         | Cam1 closed
INFO   | uav.MAVCom      | 01.732 |  mavcom.py:409 | MainThread         | MAVCom  closed


server.source_system = 222;  server.message_cnts = {0: {'BAD_DATA': 2}, 1: {'HEARTBEAT': 2}, 111: {'COMMAND_LONG': 5}}
client.source_system = 111;  client.message_cnts = {0: {'BAD_DATA': 59}, 1: {'HEARTBEAT': 114, 'TIMESYNC': 12}, 222: {'HEARTBEAT': 3, 'COMMAND_ACK': 4}}

client.source_system = 111 
 - comp.source_component = 11
 - comp.num_msgs_rcvd = 9
 - comp.num_cmds_sent = 5
 - comp.num_cmds_rcvd = 0
 - comp.num_acks_rcvd = 4
 - comp.num_acks_sent = 0
 - comp.num_acks_drop = 1
 - comp.message_cnts = {222: {'HEARTBEAT': 3, 'COMMAND_ACK': 4}, 1: {'HEARTBEAT': 2}} 

server.source_system = 222 
 - comp.source_component = 22
 - comp.num_msgs_rcvd = 3
 - comp.num_cmds_sent = 0
 - comp.num_cmds_rcvd = 2
 - comp.num_acks_rcvd = 0
 - comp.num_acks_sent = 2
 - comp.num_acks_drop = 0
 - comp.message_cnts = {111: {'COMMAND_LONG': 2}, 1: {'HEARTBEAT': 1}}
 - comp.source_component = 23
 - comp.num_msgs_rcvd = 3
 - comp.num_cmds_sent = 0
 - comp.num_cmds_rcvd = 2
 - comp.num_acks_rcvd = 0
 - com

INFO   | uav.Cli         | 02.725 | component.py:328 | MainThread         | Cli closed
INFO   | uav.MAVCom      | 02.762 |  mavcom.py:409 | MainThread         | MAVCom  closed


> For debugging help see http://localhost:3000/tutorials/mavlink_doc&debug.html and http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()